In [11]:
import psycopg2
from psycopg2 import extras
from psycopg2 import sql
from psycopg2.extensions import register_adapter, AsIs
import pandas as pd
from ast import literal_eval
import numpy
import csv

In [2]:
# Connection to server
def conectionDB(dbname, port, user, password, host):
    conn_string = "dbname={} port={} user={} password={} host={}".format(dbname, port, user, password, host)
    #print("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string)
    return conn

In [3]:
def add_data(conn, cursor, data, table):
    cursor.executemany( sql.SQL("insert into {} values ({} , {})")
        .format(sql.Identifier(table)), data)
    conn.commit()

In [4]:
path='./datasets' 
path_res='%s/BancoTGN_fix.csv' % (path)

data = pd.read_csv(path_res, sep=",", header=0)

In [5]:
# Config
dbname = 'consumptionDB'
port = '5432'
user = 'consumptionuser'
password = 'consumptionuser2020'
host = '172.250.1.22'

In [6]:
data

,Age,agency_district1,Distancia SI,Location,AMP_USD,gender,social_class
0,1.164647,-1.002122,-0.510545,0.866904,4.805797,1,9
1,-0.741557,-1.002122,-0.510545,0.866904,-0.034360,1,7
2,-0.894053,-0.219843,-0.039302,0.866904,-0.348837,2,3
3,0.783406,-1.099907,-1.497954,-0.375219,-0.121320,1,7
4,-1.351542,1.149146,-0.009277,0.866904,-0.302389,1,4
...,...,...,...,...,...,...,...
356571,0.402165,-0.024273,1.290678,-1.617341,0.013057,2,8
356572,-0.512812,0.953576,-0.713647,-0.375219,-0.375716,2,1
356573,1.088398,-1.002122,-0.510545,0.866904,-0.183093,1,6
356574,2.765857,1.051361,0.335343,0.866904,-0.323655,1,4


In [ ]:
#data1 = []
#for i in range(len(data.iloc[0:, 1:].index)):
#    row = []
#    for j in range(len(data.iloc[0:, 1:].columns)):
#        row.append(data.iloc[i, j+1])
#    data1.append(row)

In [12]:
conn1 = conectionDB(dbname, port, user, password, host)
cur1 = conn1.cursor()

with open(path_res, 'r') as data:
     reader = csv.reader(data)
     next(reader) # This skips the 1st row which is the header.
     for record in reader:
         cur1.execute(sql.SQL("insert into {} values (%s, %s, %s, %s, %s, %s, %s)")
             .format(sql.Identifier("bancotgn")), record)
         conn1.commit()
cur1.close()
conn1.close()

In [ ]:
data1[217050][5] = 0 # replace empty by zero

In [ ]:
data2= data1[len(data1)]
data2

In [ ]:
#register_adapter(numpy.int64, addapt_numpy_int64)
conn1 = conectionDB(dbname, port, user, password, host)
cur1 = conn1.cursor()

for d in data2:
    cur1.execute(sql.SQL("insert into {} values (%s, %s, %s, %s, %s, %s, %s)")
        .format(sql.Identifier("bancotgn")), d)
    conn1.commit()
cur1.statusmessage
conn1.close()

In [ ]:
conn1 = conectionDB(dbname, port, user, password, host)
cur1 = conn1.cursor()
args_str = ','.join(cur1.mogrify("(%s, %s, %s, %s, %s, %s, %s)", x) for x in data2)
cur1.execute("INSERT INTO bancotgn VALUES " + args_str)
conn1.commit()
cur1.statusmessage
conn1.close()

In [8]:
#conn1 = conectionDB(dbname, port, user, password, host)
#cur1 = conn1.cursor()
#cur1.execute(
#    sql.SQL("delete from {}")
#      .format(sql.Identifier('bancotgn')),)
#cur1.statusmessage
#conn1.commit()
#cur1.close()
#conn1.close()